### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]

In [6]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [7]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,ik,22539531
1,1,je,20769946
2,2,het,15696161
3,3,de,15258816
4,4,dat,13387137
...,...,...,...
195,195,genoeg,353128
196,196,wist,352948
197,197,oh,351988
198,198,klaar,350506


In [10]:
df_youtube_sent_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Talk Time/Data/\
1-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.csv")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
...,...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s


In [11]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 88, 157, 0, 4, 1, 36, 10, 0, 60]"
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[55, 0, 28, 0, 64, 31]"
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[20, 67, 79, 5, 35, 0, 24]"
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[53, 35, 57, 40, 12, 3]"
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 2, 5, 54, 12, 3]"
...,...,...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[16, 30, 111, 10, 56, 3]"
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[54, 63, 11, 56, 34, 3]"
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s,"[8, 10, 5, 19, 148, 15]"
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s,"[86, 16, 69, 8, 10, 58, 69]"


In [12]:
df_youtube_sent_result = df_youtube_sent_result[~df_youtube_sent_result["video_id"].isin(disable_video_id_list)]
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
51,dus ik denk niet dat hij hier nog,84,87,ben ik een beetje vroeg dus ik denk niet dat h...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=84s,"[88, 0, 90, 7, 4, 14, 37, 42]"
52,maar ik heb gewoon mijn andere,122,125,maar ik heb gewoon mijn andere jas nodig,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[18, 0, 24, 121, 31, 165]"
53,weer maar nu heb ik mijn,129,132,weer maar nu heb ik mijn eigen,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[125, 18, 54, 24, 0, 31]"
54,ik ben er zo klaar voor laten we nu,147,150,oké ik ben er zo klaar voor laten we nu wachte...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[0, 28, 19, 40, 198, 21, 102, 11, 54]"
55,nee het is voor mij de,193,195,nee het is voor mij de eerste keer volgend jaar,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[51, 2, 5, 21, 61, 3]"
...,...,...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[16, 30, 111, 10, 56, 3]"
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[54, 63, 11, 56, 34, 3]"
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s,"[8, 10, 5, 19, 148, 15]"
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s,"[86, 16, 69, 8, 10, 58, 69]"


In [13]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [14]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,dus ik denk niet dat hij hier nog,84,87,ben ik een beetje vroeg dus ik denk niet dat h...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=84s,"[88, 0, 90, 7, 4, 14, 37, 42]"
1,maar ik heb gewoon mijn andere,122,125,maar ik heb gewoon mijn andere jas nodig,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[18, 0, 24, 121, 31, 165]"
2,weer maar nu heb ik mijn,129,132,weer maar nu heb ik mijn eigen,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[125, 18, 54, 24, 0, 31]"
3,ik ben er zo klaar voor laten we nu,147,150,oké ik ben er zo klaar voor laten we nu wachte...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[0, 28, 19, 40, 198, 21, 102, 11, 54]"
4,nee het is voor mij de,193,195,nee het is voor mij de eerste keer volgend jaar,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[51, 2, 5, 21, 61, 3]"
...,...,...,...,...,...,...,...
22146,iedereen die weet als je een,579,581,afhankelijk is want iedereen die weet als je e...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[172, 20, 36, 25, 1, 6]"
22147,en we weten dat we een,606,608,als we in het bedrijf werken en we weten dat w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[8, 11, 117, 4, 11, 6]"
22148,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s,"[8, 10, 5, 19, 148, 15]"
22149,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s,"[86, 16, 69, 8, 10, 58, 69]"


In [15]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [16]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [17]:
#word_num_index_dict

In [18]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [19]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[129,
 130,
 135,
 136,
 137,
 139,
 140,
 143,
 146,
 147,
 149,
 150,
 151,
 152,
 26,
 155,
 156,
 157,
 159,
 160,
 163,
 36,
 169,
 170,
 172,
 176,
 177,
 178,
 180,
 181,
 186,
 189,
 190,
 192,
 195,
 71,
 199,
 76,
 78,
 80,
 87,
 93,
 94,
 98,
 104,
 110,
 119,
 120,
 123,
 127]

In [20]:
#Counter(index_list)

In [21]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,dus ik denk niet dat hij hier nog,84,87,ben ik een beetje vroeg dus ik denk niet dat h...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=84s,"[88, 0, 90, 7, 4, 14, 37, 42]"
1,maar ik heb gewoon mijn andere,122,125,maar ik heb gewoon mijn andere jas nodig,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[18, 0, 24, 121, 31, 165]"
2,weer maar nu heb ik mijn,129,132,weer maar nu heb ik mijn eigen,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[125, 18, 54, 24, 0, 31]"
3,ik ben er zo klaar voor laten we nu,147,150,oké ik ben er zo klaar voor laten we nu wachte...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[0, 28, 19, 40, 198, 21, 102, 11, 54]"
4,nee het is voor mij de,193,195,nee het is voor mij de eerste keer volgend jaar,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[51, 2, 5, 21, 61, 3]"
...,...,...,...,...,...,...,...
86,maken of toen zei ze nee,484,487,maken of toen zei ze nee hoor hij kan weg ik h...,B7HYu_cq_P8,https://www.youtube.com/watch?v=B7HYu_cq_P8&t=...,"[133, 65, 116, 107, 13, 51]"
87,wij veel meer moeten laten zien,39,42,misschien maar dat zouden wij veel meer moeten...,JMba7DMK3bo,https://www.youtube.com/watch?v=JMba7DMK3bo&t=39s,"[131, 113, 79, 83, 102, 95]"
88,ze echt weten waar zitten onze,77,80,zodat ze echt weten waar zitten onze rechten w...,YsseAUPtlow,https://www.youtube.com/watch?v=YsseAUPtlow&t=77s,"[13, 92, 117, 55, 187, 112]"
89,wie was nu deze man dit was,138,140,wie was nu deze man dit was djibriel in de ged...,LC4BbHLPn3U,https://www.youtube.com/watch?v=LC4BbHLPn3U&t=...,"[86, 27, 54, 81, 100, 30, 27]"


In [22]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,dus ik denk niet dat hij hier nog,84,87,ben ik een beetje vroeg dus ik denk niet dat h...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=84s,"[88, 0, 90, 7, 4, 14, 37, 42]"
1,maar ik heb gewoon mijn andere,122,125,maar ik heb gewoon mijn andere jas nodig,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[18, 0, 24, 121, 31, 165]"
2,weer maar nu heb ik mijn,129,132,weer maar nu heb ik mijn eigen,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[125, 18, 54, 24, 0, 31]"
3,ik ben er zo klaar voor laten we nu,147,150,oké ik ben er zo klaar voor laten we nu wachte...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[0, 28, 19, 40, 198, 21, 102, 11, 54]"
4,nee het is voor mij de,193,195,nee het is voor mij de eerste keer volgend jaar,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...,"[51, 2, 5, 21, 61, 3]"
...,...,...,...,...,...,...,...
86,maken of toen zei ze nee,484,487,maken of toen zei ze nee hoor hij kan weg ik h...,B7HYu_cq_P8,https://www.youtube.com/watch?v=B7HYu_cq_P8&t=...,"[133, 65, 116, 107, 13, 51]"
87,wij veel meer moeten laten zien,39,42,misschien maar dat zouden wij veel meer moeten...,JMba7DMK3bo,https://www.youtube.com/watch?v=JMba7DMK3bo&t=39s,"[131, 113, 79, 83, 102, 95]"
88,ze echt weten waar zitten onze,77,80,zodat ze echt weten waar zitten onze rechten w...,YsseAUPtlow,https://www.youtube.com/watch?v=YsseAUPtlow&t=77s,"[13, 92, 117, 55, 187, 112]"
89,wie was nu deze man dit was,138,140,wie was nu deze man dit was djibriel in de ged...,LC4BbHLPn3U,https://www.youtube.com/watch?v=LC4BbHLPn3U&t=...,"[86, 27, 54, 81, 100, 30, 27]"


In [23]:
df_youtube_sent_sample.drop("word_index", axis=1, inplace=True)

In [24]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx", index=False)

##### Selected Result Word Count

In [25]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url
0,dus ik denk niet dat hij hier nog,84,87,ben ik een beetje vroeg dus ik denk niet dat h...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=84s
1,maar ik heb gewoon mijn andere,122,125,maar ik heb gewoon mijn andere jas nodig,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
2,weer maar nu heb ik mijn,129,132,weer maar nu heb ik mijn eigen,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
3,ik ben er zo klaar voor laten we nu,147,150,oké ik ben er zo klaar voor laten we nu wachte...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
4,nee het is voor mij de,193,195,nee het is voor mij de eerste keer volgend jaar,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
...,...,...,...,...,...,...
86,maken of toen zei ze nee,484,487,maken of toen zei ze nee hoor hij kan weg ik h...,B7HYu_cq_P8,https://www.youtube.com/watch?v=B7HYu_cq_P8&t=...
87,wij veel meer moeten laten zien,39,42,misschien maar dat zouden wij veel meer moeten...,JMba7DMK3bo,https://www.youtube.com/watch?v=JMba7DMK3bo&t=39s
88,ze echt weten waar zitten onze,77,80,zodat ze echt weten waar zitten onze rechten w...,YsseAUPtlow,https://www.youtube.com/watch?v=YsseAUPtlow&t=77s
89,wie was nu deze man dit was,138,140,wie was nu deze man dit was djibriel in de ged...,LC4BbHLPn3U,https://www.youtube.com/watch?v=LC4BbHLPn3U&t=...


In [26]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [27]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,je,7
1,hij,7
2,gaat,7
3,ook,7
4,zijn,7
...,...,...
145,omdat,1
146,over,1
147,toen,1
148,twee,1


In [28]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [29]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [30]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [31]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [32]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['net',
 'zeg',
 'wordt',
 'uw',
 'dood',
 'z',
 'h',
 'mag',
 'zeker',
 'jouw',
 'hun',
 'jaar',
 'vader',
 'dag',
 'n',
 'wacht',
 'zie',
 'dacht',
 'wilde',
 'geef',
 'vrouw',
 'weet',
 'mr',
 'bedankt',
 'iedereen',
 'spijt',
 'niemand',
 'geld',
 'praten',
 'beter',
 'kon',
 'moeder',
 'zullen',
 'na',
 'genoeg',
 'm',
 'neem',
 'zal',
 'had',
 'waarom',
 'doe',
 'alleen',
 's',
 'ok',
 'terug',
 'komt',
 'tot',
 'worden',
 'tijd',
 'tegen']

In [33]:
set_not_in_sent_word = set(not_in_sent_word)

In [34]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [35]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'alleen',
 'bedankt',
 'beter',
 'dacht',
 'dag',
 'doe',
 'dood',
 'geef',
 'geld',
 'genoeg',
 'h',
 'had',
 'hun',
 'iedereen',
 'jaar',
 'jouw',
 'komt',
 'kon',
 'm',
 'mag',
 'moeder',
 'mr',
 'n',
 'na',
 'neem',
 'net',
 'niemand',
 'ok',
 'praten',
 's',
 'spijt',
 'tegen',
 'terug',
 'tijd',
 'tot',
 'uw',
 'vader',
 'vrouw',
 'waarom',
 'wacht',
 'weet',
 'wilde',
 'worden',
 'wordt',
 'z',
 'zal',
 'zeg',
 'zeker',
 'zie',
 'zullen'}

In [36]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['tegen',
 'na',
 'wordt',
 'praten',
 'jouw',
 'spijt',
 'zeg',
 'doe',
 'vrouw',
 'neem',
 'vader',
 'komt',
 'niemand',
 'weet',
 'waarom',
 'mr',
 'wilde',
 'dood',
 'h',
 'had',
 'geld',
 'mag',
 'dag',
 'm',
 's',
 'z',
 'uw',
 'bedankt',
 'tijd',
 'ok',
 'hun',
 'zullen',
 'moeder',
 'genoeg',
 'zeker',
 'beter',
 'dacht',
 'wacht',
 'geef',
 'jaar',
 'tot',
 'iedereen',
 'zie',
 'net',
 'worden',
 'n',
 'terug',
 'alleen',
 'kon',
 'zal']

In [37]:
len(not_in_sent_selected_word_list)

50

In [38]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,dus ik denk niet dat hij hier nog,84,87,ben ik een beetje vroeg dus ik denk niet dat h...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=84s
1,maar ik heb gewoon mijn andere,122,125,maar ik heb gewoon mijn andere jas nodig,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
2,weer maar nu heb ik mijn,129,132,weer maar nu heb ik mijn eigen,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
3,ik ben er zo klaar voor laten we nu,147,150,oké ik ben er zo klaar voor laten we nu wachte...,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
4,nee het is voor mij de,193,195,nee het is voor mij de eerste keer volgend jaar,HUZINsU40Fk,https://www.youtube.com/watch?v=HUZINsU40Fk&t=...
...,...,...,...,...,...,...
22146,iedereen die weet als je een,579,581,afhankelijk is want iedereen die weet als je e...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
22147,en we weten dat we een,606,608,als we in het bedrijf werken en we weten dat w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
22148,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s
22149,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s


In [39]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [40]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [41]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,tegen,ik tegen jou waar wil je mee hebben,478,481,en toen riep ik tegen jou waar wil je mee hebben,MFeFxB7xMsQ,https://www.youtube.com/watch?v=MFeFxB7xMsQ&t=...
1,tegen,je ook zien hoe dit tegen een,531,535,voor de rest is dit station erg mooi op deze m...,no_kvG64Cxk,https://www.youtube.com/watch?v=no_kvG64Cxk&t=...
2,tegen,we gaan toch goed maar zeg het maar niet tegen...,339,342,we gaan toch goed maar zeg het maar niet tegen...,l_1T3uIi4sc,https://www.youtube.com/watch?v=l_1T3uIi4sc&t=...
3,tegen,dat je er niet tegen aan,165,168,heel helder glas je mag wel oppassen dat je er...,LpIDixqRI74,https://www.youtube.com/watch?v=LpIDixqRI74&t=...
4,tegen,ook maar iets kunnen doen tegen mijn,691,695,mijn arme halina je denkt dat je vrienden ook ...,ZYh514PS3MA,https://www.youtube.com/watch?v=ZYh514PS3MA&t=...
...,...,...,...,...,...,...,...
232,zal,voor u ik zal dit niet,20,22,prettige kerstdagen hetzelfde voor u ik zal di...,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=20s
233,zal,die zie je ook nooit meer terug zo zal ik nooit,145,148,die zie je ook nooit meer terug zo zal ik nooi...,kZSGion29eo,https://www.youtube.com/watch?v=kZSGion29eo&t=...
234,zal,laat zien zal ik in deze,141,144,omdat ik al een video heb waar ik düsseldorf l...,0z1r3fhkOTw,https://www.youtube.com/watch?v=0z1r3fhkOTw&t=...
235,zal,zal ik niet laten zien in de,317,319,het treintype dat nu binnenkomt zal ik niet la...,YMGZBEoNi2A,https://www.youtube.com/watch?v=YMGZBEoNi2A&t=...


In [42]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [43]:
not_in_sent_selected_word_list_result.word.nunique()

48

In [44]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [45]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

{'mr', 'z'}

In [46]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [47]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,tegen,ik tegen jou waar wil je mee hebben,478,481,en toen riep ik tegen jou waar wil je mee hebben,MFeFxB7xMsQ,https://www.youtube.com/watch?v=MFeFxB7xMsQ&t=...
1,tegen,je ook zien hoe dit tegen een,531,535,voor de rest is dit station erg mooi op deze m...,no_kvG64Cxk,https://www.youtube.com/watch?v=no_kvG64Cxk&t=...
2,tegen,we gaan toch goed maar zeg het maar niet tegen...,339,342,we gaan toch goed maar zeg het maar niet tegen...,l_1T3uIi4sc,https://www.youtube.com/watch?v=l_1T3uIi4sc&t=...
3,tegen,dat je er niet tegen aan,165,168,heel helder glas je mag wel oppassen dat je er...,LpIDixqRI74,https://www.youtube.com/watch?v=LpIDixqRI74&t=...
4,tegen,ook maar iets kunnen doen tegen mijn,691,695,mijn arme halina je denkt dat je vrienden ook ...,ZYh514PS3MA,https://www.youtube.com/watch?v=ZYh514PS3MA&t=...
...,...,...,...,...,...,...,...
232,zal,voor u ik zal dit niet,20,22,prettige kerstdagen hetzelfde voor u ik zal di...,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=20s
233,zal,die zie je ook nooit meer terug zo zal ik nooit,145,148,die zie je ook nooit meer terug zo zal ik nooi...,kZSGion29eo,https://www.youtube.com/watch?v=kZSGion29eo&t=...
234,zal,laat zien zal ik in deze,141,144,omdat ik al een video heb waar ik düsseldorf l...,0z1r3fhkOTw,https://www.youtube.com/watch?v=0z1r3fhkOTw&t=...
235,zal,zal ik niet laten zien in de,317,319,het treintype dat nu binnenkomt zal ik niet la...,YMGZBEoNi2A,https://www.youtube.com/watch?v=YMGZBEoNi2A&t=...


In [48]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [49]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Dutch_200_Word_Group_In_Youtube_Sentence_Sample.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Count.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Sample2.xlsx']

In [50]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [51]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass